In [1]:
%matplotlib inline
import csv
import collections
import pprint
import sys
import pandas as pd
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, show
from sklearn_pandas import DataFrameMapper, cross_val_score
import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics


In [2]:
n_articles = 373

In [3]:
main_file = "/Users/wevers/Data/coca-cola_ads/1978-1989.txt" 

In [4]:
df = pd.read_csv(main_file, header=0, sep=',', names = ["date","title", "content"])
#data = data.set_index(pd.DatetimeIndex(data['paper_dc_date']))

In [5]:
df['content'] = df['content'].str.lower()
df['title'] = df['title'].str.lower()

In [6]:
df['merged'] = df['title'].map(str) + df['content'].map(str)

In [25]:
df

,date,title,content,merged
0,1978-05-27,de telegraaf,"""kêsêi hoofdprijswmnaars: l-pers. reis naar ar...","de telegraaf""kêsêi hoofdprijswmnaars: l-pers. ..."
1,1978-06-09,de telegraaf,jansen's automobielbedrijven sensationele verk...,de telegraafjansen's automobielbedrijven sensa...
2,1978-11-14,de telegraaf,exploitatie maatachappq zoekt ter overname aan...,de telegraafexploitatie maatachappq zoekt ter ...
3,1978-06-22,het vrije volk,"v \ f !.--*—'~""~— er 8e'egenheid van het visse...","het vrije volk v \ f !.--*—'~""~— er 8e'egenhei..."
4,1978-03-02,het vrije volk,schitterende elpees van o.a. diana ross o.a. c...,het vrije volk schitterende elpees van o.a. di...
5,1978-11-10,de telegraaf,"zaken en transacties 'v.. voor al uw -:c"" pvb ...",de telegraafzaken en transacties 'v.. voor al ...
6,1978-04-08,nieuwsblad van het noorden,van der ark's drankengroothandel rondeboslaan ...,nieuwsblad van het noordenvan der ark's dranke...
7,1978-05-31,limburgsch dagblad,___mm_rrllf jjr' *rv j lllii «b® 1 ifl hl? 3^s...,limburgsch dagblad___mm_rrllf jjr' *rv j lllii...
8,1978-05-10,nieuwsblad van het noorden,"fm- kl * wëm ■■■ ■ ~m^»""-^--^^^^^y— reen vroli...",nieuwsblad van het noordenfm- kl * wëm ■■■ ■ ~...
9,1978-08-26,de telegraaf,reklame-art.-stickers. gratis folder. de vries...,de telegraafreklame-art.-stickers. gratis fold...


In [39]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [41]:
g = df.groupby(df['date'].map(lambda x: x.year))

In [14]:
word_regexs = [r'\b(?:vrije\W+(?:\w+\W+){1,6}?geen|geen\W+(?:\w+\W+){1,6}?vrije)\b']

In [15]:
abs_counts = pd.Series((df['merged'].str.contains(r).sum() for r in word_regexs), word_regexs, name='abs_count')

In [16]:
abs_counts

\b(?:vrije\W+(?:\w+\W+){1,6}?geen|geen\W+(?:\w+\W+){1,6}?vrije)\b    4
Name: abs_count, dtype: int64

In [21]:
rel_counts = rel_counts.round(1)

In [22]:
wordcount = pd.concat([rel_counts, abs_counts], axis=1)

In [23]:
wordcount

,rel_count,abs_count
alcohol,0.8,3
apart*,2.7,10
aroma,1.9,7
best*,34.3,128
bijzonder*,3.5,13
[a-z]alorie*,2.4,9
chique,0.5,2
dagelijks,6.2,23
dorst,0.8,3
echte,9.1,34


In [24]:
wordcount.to_csv('keywords_1978-1989.csv') #export to csv